In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessed-data/p_suicide_data_input3_90k.csv
/kaggle/input/preprocessed-data/p_data_input2_150k.csv
/kaggle/input/preprocessed-data/p_data_100k_stop.csv
/kaggle/input/preprocessed-data/p_data_input3_211k.csv
/kaggle/input/preprocessed-data/p_data_full_stop.csv
/kaggle/input/preprocessed-data/p_suicide_data_input.csv
/kaggle/input/preprocessed-data/p_suicide_data_30k_stop.csv
/kaggle/input/mental-wellbeing-chatbot/scikitlearn/default/1/stacking_emotion_classifier.pkl
/kaggle/input/mental-wellbeing-chatbot/scikitlearn/default/1/scaler.pkl
/kaggle/input/mental-wellbeing-chatbot/scikitlearn/default/1/RandomForestClassifier_suicide_model.pkl
/kaggle/input/mental-wellbeing-chatbot/scikitlearn/default/1/word2vec_model.bin
/kaggle/input/mental-wellbeing-chatbot/scikitlearn/default/2/stacking_emotion_classifier.pkl
/kaggle/input/mental-wellbeing-chatbot/scikitlearn/default/2/scaler.pkl
/kaggle/input/mental-wellbeing-chatbot/scikitlearn/default/2/RandomForestClassifier_suicide_m

In [2]:
import pickle
import spacy
import nltk
from nltk.stem import PorterStemmer
import numpy as np
import pandas as pd
import spacy


In [3]:
nlp = spacy.load("en_core_web_sm")
nltk.download("punkt")

stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format("/kaggle/input/mental-wellbeing-chatbot/scikitlearn/default/3/GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin", binary=True)



In [5]:
with open("/kaggle/input/mental-wellbeing-chatbot/scikitlearn/default/3/emotion_model.pkl", "rb") as file:
    emotion_model = pickle.load(file)

with open("/kaggle/input/mental-wellbeing-chatbot/scikitlearn/default/3/suicide_model.pkl", "rb") as file:
    suicide_model = pickle.load(file)

In [6]:
def preprocess_tokens(text):
    doc = nlp(text)
    tokens = []
    for token in doc:
        if not token.is_punct:
            lemmatized_token = token.lemma_
            tokens.append(lemmatized_token) 
    return tokens

def vectorize(tokens, model):
    word_vectors = []
    for word in tokens:
        if word in model:
            word_vectors.append(model[word])
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)


In [7]:
emotion_to_int = {
    "happiness": 1,
    "love": 2,
    "neutral": 3,
    "sadness": 4,
    "anger": 5,
    "fear/anxiety": 6
}

In [8]:
int_to_emotion = {v: k for k, v in emotion_to_int.items()}

wellbeing_messages = {
    "happiness": (
        "When you're feeling happy, remember to share that joy with others. "
        "A genuine smile or a kind word can spread positivity all around you.\n"
        "Quote: “Happiness is not something ready-made. It comes from your own actions.” — Dalai Lama"
    ),
    "love": (
        "Love is a powerful bond—nurture it by showing appreciation and kindness to those around you. "
        "Don’t forget to practice self-love, too!\n"
        "Quote: “Love recognizes no barriers. It jumps hurdles, leaps fences, "
        "penetrates walls to arrive at its destination full of hope.” — Maya Angelou"
    ),
    "neutral": (
        "Feeling neutral is a chance to reflect and recharge. "
        "Take a moment to practice mindfulness—focus on your breath or surroundings.\n"
        "Quote: “Calmness is the cradle of power.” — J.G. Holland"
    ),
    "sadness": (
        "It’s okay to feel sad sometimes; allow yourself to process these emotions and reach out if you need support. "
        "Talking with a friend or writing in a journal can help.\n"
        "Quote: “Tears are words that need to be written.” — Paulo Coelho"
    ),
    "anger": (
        "Anger can be overwhelming. Try pausing to take a few deep breaths or stepping away for a moment. "
        "Finding healthy outlets—like exercise or journaling—can be very helpful.\n"
        "Quote: “Anger, if not restrained, is frequently more hurtful to us than the injury that provokes it.” — Seneca"
    ),
    "fear/anxiety": (
        "Anxiety can feel consuming. Practice grounding techniques—notice what you can see, hear, and feel around you. "
        "Remind yourself that you’ve overcome difficult moments before.\n"
        "Quote: “You don’t have to control your thoughts; you just have to stop letting them control you.” — Dan Millman"
    )
}


In [9]:

user_input = "I am angry and irritated all the time."
tokens = preprocess_tokens(user_input)
vector_input = vectorize(tokens, model).reshape(1, -1)

predicted_emotion_int = emotion_model.predict(vector_input)[0]


predicted_emotion_label = int_to_emotion[predicted_emotion_int]

print("Detected Emotion:", predicted_emotion_label)
print("Mental Well-being Tip & Quote:")
print(wellbeing_messages[predicted_emotion_label])

Detected Emotion: sadness
Mental Well-being Tip & Quote:
It’s okay to feel sad sometimes; allow yourself to process these emotions and reach out if you need support. Talking with a friend or writing in a journal can help.
Quote: “Tears are words that need to be written.” — Paulo Coelho


In [10]:
risk_prediction = suicide_model.predict(vector_input)[0]

if risk_prediction == 1:
    print("Suicide Risk: High")
    print("If you are in crisis, please reach out to a trusted friend, family member, or a professional.")
    print("You can also reach these trustworthy helplines:")
    print("• AASRA Helpline: +91-22-2754-6669")
    print("• Sneha Helpline (Chennai): +91-44-2464-0050")
    print("• Kiran Mental Health Helpline: 1800-599-0019 (Toll-Free)")
else:
    print("Suicide Risk: Low")
    print("If you ever feel like you need support, consider talking to a trusted friend, family member, or professional.")


Suicide Risk: Low
If you ever feel like you need support, consider talking to a trusted friend, family member, or professional.
